In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import warnings
import xgboost as xgb
import lightgbm as lgb
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv(r'D:\\BaiduNetdiskDownload\\Data\\二手车\\train.csv')
test = pd.read_csv(r'D:\\BaiduNetdiskDownload\\Data\\二手车\\test.csv')
if train.columns[0] == 'Unnamed: 0':
    train = train.drop(columns='Unnamed: 0')
if test.columns[0] == 'Unnamed: 0':
    test = test.drop(columns='Unnamed: 0')

In [ ]:
#数据预处理
def pre_deal(data):
    
    #日期处理
    data['reg_year'] = data['regDate']//10000
    data['reg_month'] = data['regDate']//100%100
    data['reg_day'] = data['regDate']%100
    data['creat_year'] = data['creatDate']//10000
    data['creat_month'] = data['creatDate']//100%100
    data['creat_day'] = data['creatDate']%100
    
    #发现异常值
    data.loc[data.power>600,'power'] = -1
    data.loc[data.notRepairedDamage == '-','notRepairedDamage'] = -1
    
    #众数填补model缺失值
    if data.model.isnull().sum()>0:
        data.model.fillna(data.model.mode()[0],inplace=True)
    
    #转换字符类型
        ##先将缺失值转化为-1，方便转成int，有小数不好看
    data = data.fillna(-1)
    need_int = ['model','bodyType','fuelType','gearbox','notRepairedDamage']
    for i in need_int:
        data[i] = data[i].astype(float).astype(int)
    
    #未缺失数据_分类特征_独热编码
    classification_f_1= ['brand']
    data[classification_f_1] = data[classification_f_1].astype(object)
    data = pd.get_dummies(data)
    
    #填补缺失值
    data = fill_rf(data)
    
    #缺失数据_分类特征_独热编码
    classification_f_2= ['fuelType','gearbox','bodyType','notRepairedDamage']
    data[classification_f_2] = data[classification_f_2].astype(object)
    data = pd.get_dummies(data)
    
    #去除无用列
    useless = ['offerType','regDate','creatDate','seller']
    data.drop(columns=useless,inplace=True)
    
    #['regionCode','model']类型过多，暂未处理
    
    return data


In [ ]:
#随机森林填补缺失值
def fill_rf(data):
    need_fill_c = ['fuelType','gearbox','bodyType','notRepairedDamage']
    need_fill_r = ['power']
    feature = [ 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14', 'reg_year', 'reg_month', 'reg_day', 'creat_year',
       'creat_month', 'creat_day', 'brand_0', 'brand_1', 'brand_2', 'brand_3',
       'brand_4', 'brand_5', 'brand_6', 'brand_7', 'brand_8', 'brand_9',
       'brand_10', 'brand_11', 'brand_12', 'brand_13', 'brand_14', 'brand_15',
       'brand_16', 'brand_17', 'brand_18', 'brand_19', 'brand_20', 'brand_21',
       'brand_22', 'brand_23', 'brand_24', 'brand_25', 'brand_26', 'brand_27',
       'brand_28', 'brand_29', 'brand_30', 'brand_31', 'brand_32', 'brand_33',
       'brand_34', 'brand_35', 'brand_36', 'brand_37', 'brand_38', 'brand_39']

    for i in need_fill_c:
        Xtrain = data[data[i] != -1][feature]
        Ytrain = data[data[i] != -1][i]
        Xtest = data[data[i] == -1][feature]
        data[data[i] == -1][i]

        rfc = RandomForestClassifier(n_estimators=100)
        rfc = rfc.fit(Xtrain, Ytrain)
        y_pre = rfc.predict(Xtest)
        data.loc[data[i] == -1,i] = y_pre

    for i in need_fill_r:
        Xtrain = data[data[i] != -1][feature]
        Ytrain = data[data[i] != -1][i]
        Xtest = data[data[i] == -1][feature]
        data[data[i] == -1][i]

        rfr = RandomForestRegressor(n_estimators=100)
        rfr = rfr.fit(Xtrain, Ytrain)
        y_pre = rfr.predict(Xtest)
        data.loc[data[i] == -1,i] = y_pre
    
    return(data)

In [ ]:
#10min
train_ = pre_deal(train)
test_ = pre_deal(test)

In [ ]:
#保存修复缺失值之后数据
train_.to_csv('D:\\BaiduNetdiskDownload\\Data\\二手车\\train_.csv')
test_.to_csv('D:\\BaiduNetdiskDownload\\Data\\二手车\\test_.csv')